In [20]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import re
encoder=LabelEncoder()

#불러오기,결측값제거
df = pd.read_csv('./list 2.csv',sep=',')
df = df.dropna()

#기업명
df["기업명"] = df["기업명"].apply(lambda x: re.sub('[|0-9기]', '', x))      # ‘| 1기’ 와 같은 기수 제외

#관심분야 더미변수화
df = df.join(df["관심분야"].str.split('/', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#우대역량 더미변수화
df["우대역량"] = df["우대역량"].str.replace(", 기타", "")                                                       #',기타'가 규칙없이 붙어있는경우가 많아서 삭제
df['우대역량'] = df['우대역량'].replace('기타', '우대역량_기타')                                                 #'기타'열이 관심분야에도 중복됨으로 교체 
df = df.join(df["우대역량"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#모집인원
df["모집 인원"]=df["모집인원"].str.extract(r'(\d+)')    #숫자만 남기기
df=df.replace({"모집 인원":'000'},'999')                #000은 세자리수를 의미함으로 999명으로 변경
df=df.replace({"모집 인원":'00'},'99')                  #00은 두자리수를 의미함으로 99명으로 변경
df=df.replace({"모집 인원":'0'},'9')                    #0은 힌자리수를 의미함으로 9명으로 변경
df=df.replace({"모집 인원":np.NaN},0)                   #NaN(미정)값 0으로 변경
df=df.astype({"모집 인원":'int'})                   
student_bins=[0,1,21,100,50000]                        #범주화  0:인원미정/1-20(명):소/20-99(명):중/99-50000(명):대
student_scale=["인원미정","소","중","대"]               
df = pd.concat([df, pd.cut(df["모집 인원"], bins=student_bins, right=False, labels=student_scale).rename("모집인원규모")], axis=1) #분포범주화

#열삭제
df=df.drop(["대외활동명","접수기간","활동지역","활동기간"],axis='columns')

#데이터저장
df=df.to_csv("test10.csv")


In [9]:
print("모집인원 기초통계량",df["모집 인원"].describe())


모집인원 기초통계량 count     362.00000
mean       44.21547
std        67.06414
min         1.00000
25%         9.00000
50%        20.00000
75%        99.00000
max      1000.00000
Name: 모집 인원, dtype: float64
